<a href="https://colab.research.google.com/github/mlindsey19/EvolComp-Rucksack/blob/master/Rucksack_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Knapsack Problem

In [0]:
import numpy as np

float_formatter = lambda x: "%.2f" % x
float_formatter1 = lambda x: "%.2f" % x
float_formatter2 = lambda x:  str(x)[0] if ( 0 <= x ) else str(x)[0:2] #for veiwing aligned columns. ONLY SHOWS FIRST DIGIT
np.set_printoptions( formatter = { 'float_kind' : float_formatter } )

L = 20
N = 10

bits = np.random.randint(low=0,high=2,size=20)
def new_bits():
  global bits
  bits = np.random.randint(low=0,high=2,size=20)
  return bits # need this return
  
vals = np.random.normal(loc=10.0, scale=2.0, size=20)
wts = np.random.normal(loc=8.0, scale=3.0, size=20)
ratio = vals/wts
cons = np.stack( [ vals, wts, ratio ] )

avg_c = wts.sum()/2
restrict_c = wts.max() * 2
D=0
CAP = avg_c

def new_constraits():
  global avg_c,restict_c, vals, wts, cons
  vals = np.random.normal(loc=10.0, scale=2.0, size=20)
  wts = np.random.normal(loc=8.0, scale=3.0, size=20)
  ratio = vals/wts
  cons = np.stack( [ vals, wts, ratio ] )
  avg_c = wts.sum()/2
  restrict_c = wts.max() * 2

pop = [ np.stack( [ new_bits(), vals, wts, ratio ]) for _ in range(N) ]
pop = np.stack(pop)

def new_pop():
  global pop
  pop = [ np.stack( [ new_bits(), vals, wts, ratio ]) for _ in range(N) ]
  pop = np.stack(pop)
  




In [0]:
def str_wt( x ):
  return ( x[0] * x[2] ).sum()
def str_val( x ):
  return ( x[0] * x[1] ).sum()


#Penalty
* **CAP** -> capacity 
* **D** -> degree of penalty ( zero=-1;  logarithmic = 0;  linear = 1; quadratic = 2 )





In [0]:
pen = ratio.max()
def set_pen():
  global pen
  pen = ratio.max()


def penalty(sw):
  if( sw > CAP ):
    global pen
    p = ( sw - CAP ) * pen
    p = np.log2( 1 + p) if ( D == 0 ) else p #assumes only positive values of p
    p = p*p if (D == 2) else p
    return 0 if ( D == -1 ) else p
  else:
    return 0
def new_problem():
  new_constraits()
  new_pop()
  set_pen()

#Repair


In [0]:
def repair():
  global pop
  for i in range(10):
    if ( str_wt(pop[i]) > CAP ):
      while( True ):
        while ( True ):
          k = np.random.randint(0, high = 20)
          if ( pop[i,0, k ] == 1 ):
            pop[ i ,0, k ] = 0
            break
        if ( str_wt( pop[i] ) <= CAP or ( not pop[i,0].sum() ) ):
          break

#Selection



In [0]:
new_problem()

In [0]:
def fit_vals():
  global pop
  y = np.stack( [ str_val(pop[i]) for i in range(N)])
  w = [str_wt(pop[ i ]) for i in range(N)]
  f = [ penalty(i ) for i in w ]
  av = y - f
  return np.stack(av)
def fit_wts():
  global pop
  y =  [ str_wt(pop[i]) for i in range(N)]
  return np.stack(y)

def sort_idx():
  return fit_vals().argsort()

In [0]:
linRank = np.empty(N)
def set_linRank(min):
  global linRank
  max = 2 - min
  Plr = lambda r:( min + ( r - 1 ) / ( N - 1 ) * ( max - min ) ) / N
  PlrMap = np.array( [ Plr(x) for x in range( 1, 11 ) ] ) 
  for i in range(N):
    if (i == 0):
      linRank[ i ] = PlrMap[ i ]
    else:
      linRank[ i ] = PlrMap[ i ] + linRank[ i - 1 ]
set_linRank(0.0)

In [0]:
def selectionLR():
  global pop
  newPop = np.empty( (N, 4, L ) )
  si = sort_idx()
  for i in range( N ):
    s = np.random.uniform( 0, 1 )
    for idx, c in enumerate(linRank):
      if( s <= c ):
        newPop[ i ] = pop[ si[ idx ] ]
        break
  pop = newPop

In [0]:
Psel = 0.7
def selectionBT():
  global pop
  newPop = np.empty( ( N, 4, L ) )
  for i in range(N):
    s1 = np.random.randint(0,high=10)    
    s2 = np.random.randint(0,high=10)
    b = np.take( pop[:,0,:], s1, axis=0 )
    a = np.take( pop[:,0,:], s2, axis=0 )
    fit = fit_vals()
    p = np.random.uniform( 0, 1 )
    if ( fit[s1] > fit[s2] and p < Psel ):
      newPop[i] = pop[s1]
    else:
      newPop[i] = pop[s2]
  pop = newPop

#Crossover

In [0]:
Pcross = 0.6

def crossover_2pt():
  global pop
  newPop = np.empty( ( N, 4, L ) )
  np.random.shuffle(pop)
  for i in range( N // 2 ):
    if ( np.random.uniform( 0, 1 ) < Pcross ):
      j = 2 * i
      pt1 = np.random.randint(0,high=20) 
      pt2 = np.random.randint(pt1,high=20)
      pop[ j, 0, pt1:pt2 ], pop[ j + 1, 0, pt1:pt2 ] = pop[ j + 1, 0, pt1:pt2 ].copy(), pop[ j, 0, pt1:pt2 ].copy()
      
def crossover_1pt():
  global pop
  newPop = np.empty( ( N, 4, L ) )
  np.random.shuffle(pop)
  for i in range( N // 2 ):
    if ( np.random.uniform( 0, 1 ) < Pcross ):
      j = 2 * i
      pt1 = np.random.randint(0,high=20) 
      pop[ j, 0, pt1: ], pop[ j + 1, 0, :pt1 ] = pop[ j + 1, 0, pt1: ].copy(), pop[ j, 0, :pt1 ].copy()

def crossover_and():
  global pop
  np.random.shuffle(pop)
  newPop = np.empty( ( N, 4, L ) )
  for i in range( N ):
    if ( np.random.uniform( 0, 1 ) < Pcross ):
      r = np.random.randint(0,high=10)
      b = np.take( pop[:,0,:], i, axis=0 )
      a = np.take( pop[:,0,:], r, axis=0 )
      newPop[i,0] = ( a.astype(bool) & b.astype(bool) ).astype(int)
    else:
      newPop[i,0] = pop[i,0]


#Mutation

In [0]:
Pmut = 0.01
def mutation():
  mutMatrix = np.random.random_sample((10,20))
  mm = (mutMatrix < Pmut).astype(int)
  pop[:,0,:] = (pop[:,0,:] + mm) % 2

#Evolution

In [11]:
new_problem()
CAP = avg_c
D=-1
np.set_printoptions( formatter = { 'float_kind' : float_formatter } )

print(fit_vals())
D=0

print(fit_vals())
D=1

print(fit_vals())

print(np.array([CAP]))

print(fit_wts())
D=2

for _ in range(100):
  selectionLR()
  crossover_2pt()
  mutation()
# D=1
# for _ in range(100):
#   selectionBT()
#   crossover_2pt()
#   mutation()
# D=2
# for _ in range(30):
#   selectionBT()
#   crossover_and()
#   mutation()
# D=2
# for _ in range(30):
#   selectionBT()
#   crossover_and()
#   mutation()
#   repair()

D=2
print("val",fit_vals())
print(np.array([CAP]))
print("wts",fit_wts())

np.set_printoptions( formatter = { 'float_kind' : float_formatter2 } )

print( pop[:,0] )

[118.04 107.24 69.02 102.83 122.63 93.40 125.71 76.78 94.88 103.32]
[112.37 102.97 69.02 102.83 117.89 93.40 120.62 76.78 91.70 103.32]
[67.97 88.90 69.02 102.83 96.78 93.40 92.51 76.78 86.78 103.32]
[86.67]
[107.33 94.24 55.47 75.90 97.34 76.23 100.37 77.69 90.01 83.70]
val [92.13 113.04 -119.65 -93.97 113.04 113.04 118.83 113.04 102.81 101.53]
[86.67]
wts [73.10 86.87 93.10 92.78 86.87 86.87 86.93 86.87 79.34 76.25]
[[1 0 0 0 1 0 1 1 0 0 1 0 1 0 1 0 0 1 1 0]
 [1 0 0 0 1 1 1 1 0 0 1 0 1 0 1 0 1 1 1 0]
 [1 0 0 0 1 1 1 1 0 1 1 0 1 0 1 0 1 1 1 0]
 [1 0 0 0 1 1 1 1 1 0 1 0 1 0 1 0 1 1 1 0]
 [1 0 0 0 1 1 1 1 0 0 1 0 1 0 1 0 1 1 1 0]
 [1 0 0 0 1 1 1 1 0 0 1 0 1 0 1 0 1 1 1 0]
 [1 0 0 0 1 1 1 0 1 0 1 0 1 0 1 0 1 1 1 0]
 [1 0 0 0 1 1 1 1 0 0 1 0 1 0 1 0 1 1 1 0]
 [1 0 0 0 1 1 1 1 0 0 1 0 1 0 1 0 0 1 1 0]
 [1 0 0 0 1 1 1 1 0 0 1 0 1 0 1 0 1 0 1 0]]


In [22]:
new_problem()
CAP = restrict_c
D=-1
np.set_printoptions( formatter = { 'float_kind' : float_formatter } )

print(fit_vals())
D=0

print(fit_vals())
D=1

print(fit_vals())

print(np.array([CAP]))

print(fit_wts())
D=0

for _ in range(30):
  selectionLR()
  crossover_1pt()
  mutation()
D=1
for _ in range(20):
  selectionBT()
  crossover_2pt()
  mutation()
D=1
for _ in range(20):
  selectionBT()
  crossover_and()
  mutation()
D=2
for _ in range(10):
  selectionBT()
  crossover_and()
  mutation()
  repair()

D=2
print("val",fit_vals())
print(np.array([CAP]))
print("wts",fit_wts())

np.set_printoptions( formatter = { 'float_kind' : float_formatter2 } )

print(pop[:,0])


[156.04 84.19 118.83 81.24 118.06 107.50 103.52 110.89 130.50 121.19]
[148.26 77.70 111.21 74.43 110.94 100.50 96.56 103.92 122.95 113.71]
[-62.46 -4.59 -76.88 -30.03 -20.50 -19.40 -19.83 -13.41 -55.57 -55.57]
[22.58]
[112.71 59.20 103.31 68.48 79.74 74.93 73.46 73.85 99.33 95.49]
val [51.86 51.86 53.85 40.28 32.12 51.86 33.59 51.86 56.34 51.86]
[22.58]
wts [21.41 21.41 20.39 17.14 12.47 21.41 13.57 21.41 16.95 21.41]
[[1 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 0 0 1 0 1 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0]]
